參考文章: https://matthewpalmer.net/kubernetes-app-developer/articles/kubernetes-networking-guide-beginners.html

參考文章: https://kubernetes.io/docs/concepts/cluster-administration/networking/

參考文章: https://kubernetes.io/docs/concepts/services-networking/dns-pod-service/

參考文章: https://kubernetes.io/docs/concepts/services-networking/service/

# Cluster Networking

# Kubernetes Networking Guide for Beginners

<img src='./img/35.png'>

# Communication between containers in the same pod

### What is a network namespace?

<img src='./img/same-pod.gif'>

# Communication between pods on the same node

### What is a Network Bridge?

<img src='./img/pods-on-node.gif'>

### 範例

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: mypod1
spec:
  nodeName: 'k8s-worker1'
  containers:
  - name: pod1
    image: xiaopeng163/net-box
    command: ["sh", "-c", "while true; do echo $(date) >> /tmp/index.html; sleep 60; done"]
---
apiVersion: v1
kind: Pod
metadata:
  name: mypod2
spec:
  nodeName: 'k8s-worker1'
  containers:
  - name: pod2
    image: xiaopeng163/net-box
    command: ["sh", "-c", "while true; do echo $(date) >> /tmp/index.html; sleep 60; done"]

# Communication between pods on different nodes

<img src='./img/node-to-node.gif'>

### 範例

參考文章: https://medium.com/@anilkreddyr/kubernetes-with-flannel-understanding-the-networking-part-2-78b53e5364c7

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: mypod1
spec:
  containers:
  - name: pod1
    image: xiaopeng163/net-box
    command: ["sh", "-c", "while true; do echo $(date) >> /tmp/index.html; sleep 60; done"]
---
apiVersion: v1
kind: Pod
metadata:
  name: mypod2
spec:
  containers:
  - name: pod2
    image: xiaopeng163/net-box
    command: ["sh", "-c", "while true; do echo $(date) >> /tmp/index.html; sleep 60; done"]

<img src='./img/36.png'>

# DNS for Services and Pods

### Namespaces of Services

# Services

### A/AAAA records

### SRV records

# Pods

### A/AAAA records 

### Pod's hostname and subdomain fields

### 範例

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: default-subdomain
spec:
  selector:
    name: busybox
  clusterIP: None
  ports:
  - name: foo # Actually, no port is needed.
    port: 1234
    targetPort: 1234
---
apiVersion: v1
kind: Pod
metadata:
  name: busybox1
  labels:
    name: busybox
spec:
  hostname: busybox-1
  subdomain: default-subdomain
  containers:
  - image: busybox:1.28
    command:
      - sleep
      - "3600"
    name: busybox
---
apiVersion: v1
kind: Pod
metadata:
  name: busybox2
  labels:
    name: busybox
spec:
  hostname: busybox-2
  subdomain: default-subdomain
  containers:
  - image: busybox:1.28
    command:
      - sleep
      - "3600"
    name: busybox

### Pod's setHostnameAsFQDN field

### Pod's DNS Policy

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: busybox
  namespace: default
spec:
  containers:
  - image: busybox:1.28
    command:
      - sleep
      - "3600"
    imagePullPolicy: IfNotPresent
    name: busybox
  restartPolicy: Always
  hostNetwork: true
  dnsPolicy: ClusterFirstWithHostNet

### Pod's DNS Config

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  namespace: default
  name: dns-example
spec:
  containers:
    - name: test
      image: nginx
  dnsPolicy: "None"
  dnsConfig:
    nameservers:
      - 1.2.3.4
    searches:
      - ns1.svc.cluster-domain.example
      - my.dns.search.suffix
    options:
      - name: ndots
        value: "2"
      - name: edns0

In [ ]:
nameserver 1.2.3.4
search ns1.svc.cluster-domain.example my.dns.search.suffix
options ndots:2 edns0

In [ ]:
kubectl exec -it dns-example -- cat /etc/resolv.conf

### 範例

In [ ]:
kubectl get pod -A

In [ ]:
kubectl get deployment -A

In [ ]:
kubectl get replicasets -A

In [ ]:
kubectl get service -A

<img src='./img/37.png'>

In [ ]:
kubectl run client --image=xiaopeng163/net-box --command -- sh -c "sleep 100000"

In [ ]:
kubectl get pods

In [ ]:
kubectl exec -it client -- sh 

In [ ]:
cat /etc/resolv.conf

<img src='./img/38.png'>

In [ ]:
ip addr

In [ ]:
nslookup 172-17-0-4.default.pod.cluster.local

<img src='./img/39.png'>

In [ ]:
kubectl get configeMap -A

In [ ]:
kubectl describe configmaps coredns --namespace kube-system

<img src='./img/40.png'>

# Service

<img src='./img/service.gif'>

# Motivation 

# Service resources

### Cloud-native service discovery

# Defining a Service

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-service
spec:
  selector:
    app.kubernetes.io/name: MyApp
  ports:
    - protocol: TCP
      port: 80
      targetPort: 9376

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: nginx
  labels:
    app.kubernetes.io/name: proxy
spec:
  containers:
  - name: nginx
    image: nginx:stable
    ports:
      - containerPort: 80
        name: http-web-svc

---
apiVersion: v1
kind: Service
metadata:
  name: nginx-service
spec:
  selector:
    app.kubernetes.io/name: proxy
  ports:
  - name: name-of-service-port
    protocol: TCP
    port: 80
    targetPort: http-web-svc

### Services without selectors

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-service
spec:
  ports:
    - protocol: TCP
      port: 80
      targetPort: 9376

In [ ]:
apiVersion: discovery.k8s.io/v1
kind: EndpointSlice
metadata:
  name: my-service-1 # by convention, use the name of the Service
                     # as a prefix for the name of the EndpointSlice
  labels:
    # You should set the "kubernetes.io/service-name" label.
    # Set its value to match the name of the Service
    kubernetes.io/service-name: my-service
addressType: IPv4
ports:
  - name: '' # empty because port 9376 is not assigned as a well-known
             # port (by IANA)
    appProtocol: http
    protocol: TCP
    port: 9376
endpoints:
  - addresses:
      - "10.4.5.6" # the IP addresses in this list can appear in any order
      - "10.1.2.3"

### Custom EndpointSlices

### Endpoints

##### Over-capacity endpoints 

##### Application protocol 

# EndpointSlices

### EndpointSlice API

In [ ]:
apiVersion: discovery.k8s.io/v1
kind: EndpointSlice
metadata:
  name: example-abc
  labels:
    kubernetes.io/service-name: example
addressType: IPv4
ports:
  - name: http
    protocol: TCP
    port: 80
endpoints:
  - addresses:
      - "10.1.2.3"
    conditions:
      ready: true
    hostname: pod-1
    nodeName: node-1
    zone: us-west2-a

### Address types

### Conditions

##### Ready

##### Serving

##### Terminating

### Topology information

### Comparison with Endpoints

# Virtual IPs and service proxies

### Why not use round-robin DNS?

### Configuration

### User space proxy mode

<img src='./img/2.svg'>

###  iptables proxy mode

<img src='./img/3.svg'>

### IPVS proxy mode

<img src='./img/4.svg'>

# Multi-Port Services

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-service
spec:
  selector:
    app.kubernetes.io/name: MyApp
  ports:
    - name: http
      protocol: TCP
      port: 80
      targetPort: 9376
    - name: https
      protocol: TCP
      port: 443
      targetPort: 9377

# Choosing your own IP address

# Traffic policies

### Internal traffic policy

# <span style="color:red">Discovering services</span>

##### Environment variables

##### DNS

### 範例: 使用 DNS

In [ ]:
kubectl create deployment demo --image=gcr.io/google-samples/hello-app:1.0 --port=8080

In [ ]:
kubectl expose deployment demo

In [ ]:
kubectl get svc --namespace kube-system

In [ ]:
nslookup demo.default.svc.cluster.local 10.96.0.10

In [ ]:
kubectl run client --image=xiaopeng163/net-box --command -- sh -c "sleep 100000"

In [ ]:
kubectl exec -it client -- sh

In [ ]:
nslookup demo.default.svc.cluster.local
curl demo.default.svc.cluster.local:8080

In [ ]:
nslookup demo
curl demo:8080

### 範例: 使用環境變數

In [ ]:
kubectl run client --image=xiaopeng163/net-box --command -- sh -c "sleep 100000"

In [ ]:
kubectl exec -it client -- sh

In [ ]:
查看所有的環境變數，會看到只要創建 service，就會幫我們創建對應的一些環境變數。

In [ ]:
env

### 範例

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: redis-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      app: redis-server
  template:
    metadata:
      labels:
        app: redis-server
    spec:
      containers:
        - name: redis-server
          image: redis:latest
          command:
            - redis-server
            - --requirepass
            - redis
          ports:
            - containerPort: 6379
---
apiVersion: v1
kind: Service
metadata:
  name: redis
spec:
  selector:
    app: redis-server
  ports:
    - protocol: TCP
      port: 6379
      targetPort: 6379

---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: flask-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      app: flask-app
  template:
    metadata:
      labels:
        app: flask-app
    spec:
      containers:
        - name: flask-app
          image: xiaopeng163/flask-redis
          ports:
            - containerPort: 5000
          env:
            - name: REDIS_HOST
              value: ????????????????????????
---
apiVersion: v1
kind: Service
metadata:
  name: flask-service
spec:
  selector:
    app: flask-app
  ports:
    - protocol: TCP
      port: 8000
      targetPort: 5000

In [ ]:
kubectl apply -f flask_redis.yml

In [ ]:
kubectl get service

In [ ]:
docker container exec -it 5aa8 sh

In [ ]:
curl 10.105.109.42:8000

# Headless Services

### With selectors 

### Without selectors

# <span style='color:red'>Publishing Services (ServiceTypes)</span>

參考影片: https://www.youtube.com/watch?v=fXQbkW1RNhE&t=57s

### Type ClusterIP

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-world
spec:
  replicas: 1
  selector:
    matchLabels:
      app: hello-world
  template:
    metadata:
      labels:
        app: hello-world
    spec:
      containers:
      - name: hello-world
        image: gcr.io/google-samples/hello-app:1.0
        ports:
        - containerPort: 8080

In [ ]:
kubectl apply -f hello-world.yml 

In [ ]:
kubectl get deployment

In [ ]:
kubectl scale deployment hello-world --replicas=2

In [ ]:
kubectl expose deployment hello-world --port=80 --target-port=8080

In [ ]:
kubectl get svc

In [ ]:
curl clusterIP:port

In [ ]:
kubectl get svc hello-world -o yaml

<img src='./img/41.png'>

In [ ]:
sudo iptables -t nat -L KUBE-SERVICES -n  | column -t

<img src='./img/42.png'>

In [ ]:
sudo iptables -t nat -L KUBE-SVC-DZ6LTOHRG6HQWHYE -n  | column -t

<img src='./img/43.png'>

### Type headless

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: headless-deployment
spec:
  replicas: 3
  selector:
    matchLabels:
      app: headless-demo
  template:
    metadata:
      labels:
        app: headless-demo
    spec:
      containers:
      - name: headless-demo
        image: gcr.io/google-samples/hello-app:1.0
        ports:
        - containerPort: 8080

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: headless-service
spec:
  type: ClusterIP
  ClusterIP: None
  selector:
    app: headless-demo
  ports:
    - protocol: TCP
      port: 80
      targetPort: 8080

In [ ]:
kubectl describe service headless-service

<img src='./img/44.png'>

In [ ]:
nslookup headless-service.default.svc.cluster.local

<img src='./img/45.png'>

### Type NodePort

##### Choosing your own port

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-service
spec:
  type: NodePort
  selector:
    app.kubernetes.io/name: MyApp
  ports:
      # By default and for convenience, the `targetPort` is set to the same value as the `port` field.
    - port: 80
      targetPort: 80
      # Optional field
      # By default and for convenience, the Kubernetes control plane will allocate a port from a range (default: 30000-32767)
      nodePort: 30007

##### Custom IP address configuration for type: NodePort Services

### 範例

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nodeport-deployment
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nodeport-demo
  template:
    metadata:
      labels:
        app: nodeport-demo
    spec:
      containers:
      - name: nodeport-demo
        image: gcr.io/google-samples/hello-app:1.0
        ports:
        - containerPort: 8080

In [ ]:
kubectl expose deployment nodeport-deployment --target-port=8080 --type=NodePort

In [ ]:
kubectl get service

<img src='./img/46.png'>

In [ ]:
curl 10.109.58.172:8080

In [ ]:
curl 192.168.49.2:32540

### Type LoadBalancer

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-service
spec:
  selector:
    app.kubernetes.io/name: MyApp
  ports:
    - protocol: TCP
      port: 80
      targetPort: 9376
  clusterIP: 10.0.171.239
  type: LoadBalancer
status:
  loadBalancer:
    ingress:
    - ip: 192.0.2.127

### Type ExternalName

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-service
  namespace: prod
spec:
  type: ExternalName
  externalName: my.database.example.com

### 範例

參考文章：https://www.jianshu.com/p/758cfafcf80d

參考文章: https://www.sobyte.net/post/2022-04/kubernetes-ext-service/

<img src='./img/1.webp'>

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: baidu-service
spec:
  type: ExternalName
  externalName: www.baidu.com

In [ ]:
kubectl apply -f baidu-service.yaml

In [ ]:
kubectl get sercice
kubectl describe service baidu-service

In [ ]:
@FeignClient(value = "feignClient", url = "http://baidu-service")

### 範例

In [ ]:
apiVersion: v1
kind: Endpoints
metadata:
  name: custom-endpoint
subsets:
  - addresses:
      - ip: 192.168.3.175
    ports:
      - port: 8091

In [ ]:
kubectl get endpoints

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: custom-endpoint
spec:
  clusterIP: None
  type: ClusterIP
  ports:
  - port: 8091
    targetPort: 8091

In [ ]:
kubectl get service

In [ ]:
kubectl describe service custom-endpoint

<img src='./img/47.png'>

In [ ]:
kubectl exec -it client -- sh
nslookup custom-endpoint

<img src='./img/48.png'>

### External IPs

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-service
spec:
  selector:
    app.kubernetes.io/name: MyApp
  ports:
    - name: http
      protocol: TCP
      port: 80
      targetPort: 9376
  externalIPs:
    - 80.11.12.10

# Shortcomings

# Virtual IP implementation

### Avoiding collisions

### IP address ranges for type: ClusterIP Services

### Service IP addresses

##### Userspace

##### iptables

##### IPVS

### 範例: Connecting Applications with Services

參考文章: https://kubernetes.io/docs/tutorials/services/connect-applications-service/

##### The Kubernetes model for connecting containers

##### Exposing pods to the cluster

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-nginx
spec:
  selector:
    matchLabels:
      run: my-nginx
  replicas: 2
  template:
    metadata:
      labels:
        run: my-nginx
    spec:
      containers:
      - name: my-nginx
        image: nginx
        ports:
        - containerPort: 80

In [ ]:
kubectl apply -f ./run-my-nginx.yaml
kubectl get pods -l run=my-nginx -o wide

##### Creating a Service

In [ ]:
kubectl expose deployment/my-nginx

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-nginx
  labels:
    run: my-nginx
spec:
  ports:
  - port: 80
    protocol: TCP
  selector:
    run: my-nginx

In [ ]:
kubectl describe svc my-nginx

In [ ]:
kubectl get endpointslices -l kubernetes.io/service-name=my-nginx

### Accessing the Service

##### Environment Variables

In [ ]:
kubectl exec my-nginx-3800858182-jr4a2 -- printenv | grep SERVICE

In [ ]:
KUBERNETES_SERVICE_HOST=10.0.0.1
KUBERNETES_SERVICE_PORT=443
KUBERNETES_SERVICE_PORT_HTTPS=443

In [ ]:
kubectl scale deployment my-nginx --replicas=0; kubectl scale deployment my-nginx --replicas=2;
kubectl get pods -l run=my-nginx -o wide

In [ ]:
kubectl exec my-nginx-3800858182-e9ihh -- printenv | grep SERVICE

##### DNS

In [ ]:
kubectl get services kube-dns --namespace=kube-system

In [ ]:
kubectl run curl --image=radial/busyboxplus:curl -i --tty

### Securing the Service

https example: https://github.com/kubernetes/examples/tree/master/staging/https-nginx/

In [ ]:
make keys KEY=/tmp/nginx.key CERT=/tmp/nginx.crt
kubectl create secret tls nginxsecret --key /tmp/nginx.key --cert /tmp/nginx.crt

In [ ]:
kubectl get secrets

In [ ]:
kubectl create configmap nginxconfigmap --from-file=default.conf

In [ ]:
kubectl get configmaps

In [ ]:
apiVersion: "v1"
kind: "Secret"
metadata:
  name: "nginxsecret"
  namespace: "default"
type: kubernetes.io/tls
data:
  tls.crt: "LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURIekNDQWdlZ0F3SUJBZ0lKQUp5M3lQK0pzMlpJTUEwR0NTcUdTSWIzRFFFQkJRVUFNQ1l4RVRBUEJnTlYKQkFNVENHNW5hVzU0YzNaak1SRXdEd1lEVlFRS0V3aHVaMmx1ZUhOMll6QWVGdzB4TnpFd01qWXdOekEzTVRKYQpGdzB4T0RFd01qWXdOekEzTVRKYU1DWXhFVEFQQmdOVkJBTVRDRzVuYVc1NGMzWmpNUkV3RHdZRFZRUUtFd2h1CloybHVlSE4yWXpDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBSjFxSU1SOVdWM0IKMlZIQlRMRmtobDRONXljMEJxYUhIQktMSnJMcy8vdzZhU3hRS29GbHlJSU94NGUrMlN5ajBFcndCLzlYTnBwbQppeW1CL3JkRldkOXg5UWhBQUxCZkVaTmNiV3NsTVFVcnhBZW50VWt1dk1vLzgvMHRpbGhjc3paenJEYVJ4NEo5Ci82UVRtVVI3a0ZTWUpOWTVQZkR3cGc3dlVvaDZmZ1Voam92VG42eHNVR0M2QURVODBpNXFlZWhNeVI1N2lmU2YKNHZpaXdIY3hnL3lZR1JBRS9mRTRqakxCdmdONjc2SU90S01rZXV3R0ljNDFhd05tNnNTSzRqYUNGeGpYSnZaZQp2by9kTlEybHhHWCtKT2l3SEhXbXNhdGp4WTRaNVk3R1ZoK0QrWnYvcW1mMFgvbVY0Rmo1NzV3ajFMWVBocWtsCmdhSXZYRyt4U1FVQ0F3RUFBYU5RTUU0d0hRWURWUjBPQkJZRUZPNG9OWkI3YXc1OUlsYkROMzhIYkduYnhFVjcKTUI4R0ExVWRJd1FZTUJhQUZPNG9OWkI3YXc1OUlsYkROMzhIYkduYnhFVjdNQXdHQTFVZEV3UUZNQU1CQWY4dwpEUVlKS29aSWh2Y05BUUVGQlFBRGdnRUJBRVhTMW9FU0lFaXdyMDhWcVA0K2NwTHI3TW5FMTducDBvMm14alFvCjRGb0RvRjdRZnZqeE04Tzd2TjB0clcxb2pGSW0vWDE4ZnZaL3k4ZzVaWG40Vm8zc3hKVmRBcStNZC9jTStzUGEKNmJjTkNUekZqeFpUV0UrKzE5NS9zb2dmOUZ3VDVDK3U2Q3B5N0M3MTZvUXRUakViV05VdEt4cXI0Nk1OZWNCMApwRFhWZmdWQTRadkR4NFo3S2RiZDY5eXM3OVFHYmg5ZW1PZ05NZFlsSUswSGt0ejF5WU4vbVpmK3FqTkJqbWZjCkNnMnlwbGQ0Wi8rUUNQZjl3SkoybFIrY2FnT0R4elBWcGxNSEcybzgvTHFDdnh6elZPUDUxeXdLZEtxaUMwSVEKQ0I5T2wwWW5scE9UNEh1b2hSUzBPOStlMm9KdFZsNUIyczRpbDlhZ3RTVXFxUlU9Ci0tLS0tRU5EIENFUlRJRklDQVRFLS0tLS0K"
  tls.key: "LS0tLS1CRUdJTiBQUklWQVRFIEtFWS0tLS0tCk1JSUV2UUlCQURBTkJna3Foa2lHOXcwQkFRRUZBQVNDQktjd2dnU2pBZ0VBQW9JQkFRQ2RhaURFZlZsZHdkbFIKd1V5eFpJWmVEZWNuTkFhbWh4d1NpeWF5N1AvOE9ta3NVQ3FCWmNpQ0RzZUh2dGtzbzlCSzhBZi9WemFhWm9zcApnZjYzUlZuZmNmVUlRQUN3WHhHVFhHMXJKVEVGSzhRSHA3VkpMcnpLUC9QOUxZcFlYTE0yYzZ3MmtjZUNmZitrCkU1bEVlNUJVbUNUV09UM3c4S1lPNzFLSWVuNEZJWTZMMDUrc2JGQmd1Z0ExUE5JdWFubm9UTWtlZTRuMG4rTDQKb3NCM01ZUDhtQmtRQlAzeE9JNHl3YjREZXUraURyU2pKSHJzQmlIT05Xc0RadXJFaXVJMmdoY1kxeWIyWHI2UAozVFVOcGNSbC9pVG9zQngxcHJHclk4V09HZVdPeGxZZmcvbWIvNnBuOUYvNWxlQlkrZStjSTlTMkQ0YXBKWUdpCkwxeHZzVWtGQWdNQkFBRUNnZ0VBZFhCK0xkbk8ySElOTGo5bWRsb25IUGlHWWVzZ294RGQwci9hQ1Zkank4dlEKTjIwL3FQWkUxek1yall6Ry9kVGhTMmMwc0QxaTBXSjdwR1lGb0xtdXlWTjltY0FXUTM5SjM0VHZaU2FFSWZWNgo5TE1jUHhNTmFsNjRLMFRVbUFQZytGam9QSFlhUUxLOERLOUtnNXNrSE5pOWNzMlY5ckd6VWlVZWtBL0RBUlBTClI3L2ZjUFBacDRuRWVBZmI3WTk1R1llb1p5V21SU3VKdlNyblBESGtUdW1vVlVWdkxMRHRzaG9reUxiTWVtN3oKMmJzVmpwSW1GTHJqbGtmQXlpNHg0WjJrV3YyMFRrdWtsZU1jaVlMbjk4QWxiRi9DSmRLM3QraTRoMTVlR2ZQegpoTnh3bk9QdlVTaDR2Q0o3c2Q5TmtEUGJvS2JneVVHOXBYamZhRGR2UVFLQmdRRFFLM01nUkhkQ1pKNVFqZWFKClFGdXF4cHdnNzhZTjQyL1NwenlUYmtGcVFoQWtyczJxWGx1MDZBRzhrZzIzQkswaHkzaE9zSGgxcXRVK3NHZVAKOWRERHBsUWV0ODZsY2FlR3hoc0V0L1R6cEdtNGFKSm5oNzVVaTVGZk9QTDhPTm1FZ3MxMVRhUldhNzZxelRyMgphRlpjQ2pWV1g0YnRSTHVwSkgrMjZnY0FhUUtCZ1FEQmxVSUUzTnNVOFBBZEYvL25sQVB5VWs1T3lDdWc3dmVyClUycXlrdXFzYnBkSi9hODViT1JhM05IVmpVM25uRGpHVHBWaE9JeXg5TEFrc2RwZEFjVmxvcG9HODhXYk9lMTAKMUdqbnkySmdDK3JVWUZiRGtpUGx1K09IYnRnOXFYcGJMSHBzUVpsMGhucDBYSFNYVm9CMUliQndnMGEyOFVadApCbFBtWmc2d1BRS0JnRHVIUVV2SDZHYTNDVUsxNFdmOFhIcFFnMU16M2VvWTBPQm5iSDRvZUZKZmcraEppSXlnCm9RN3hqWldVR3BIc3AyblRtcHErQWlSNzdyRVhsdlhtOElVU2FsbkNiRGlKY01Pc29RdFBZNS9NczJMRm5LQTQKaENmL0pWb2FtZm1nZEN0ZGtFMXNINE9MR2lJVHdEbTRpb0dWZGIwMllnbzFyb2htNUpLMUI3MkpBb0dBUW01UQpHNDhXOTVhL0w1eSt5dCsyZ3YvUHM2VnBvMjZlTzRNQ3lJazJVem9ZWE9IYnNkODJkaC8xT2sybGdHZlI2K3VuCnc1YytZUXRSTHlhQmd3MUtpbGhFZDBKTWU3cGpUSVpnQWJ0LzVPbnlDak9OVXN2aDJjS2lrQ1Z2dTZsZlBjNkQKckliT2ZIaHhxV0RZK2Q1TGN1YSt2NzJ0RkxhenJsSlBsRzlOZHhrQ2dZRUF5elIzT3UyMDNRVVV6bUlCRkwzZAp4Wm5XZ0JLSEo3TnNxcGFWb2RjL0d5aGVycjFDZzE2MmJaSjJDV2RsZkI0VEdtUjZZdmxTZEFOOFRwUWhFbUtKCnFBLzVzdHdxNWd0WGVLOVJmMWxXK29xNThRNTBxMmk1NVdUTThoSDZhTjlaMTltZ0FGdE5VdGNqQUx2dFYxdEYKWSs4WFJkSHJaRnBIWll2NWkwVW1VbGc9Ci0tLS0tRU5EIFBSSVZBVEUgS0VZLS0tLS0K"

In [ ]:
kubectl apply -f nginxsecrets.yaml
kubectl get secrets

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-nginx
  labels:
    run: my-nginx
spec:
  type: NodePort
  ports:
  - port: 8080
    targetPort: 80
    protocol: TCP
    name: http
  - port: 443
    protocol: TCP
    name: https
  selector:
    run: my-nginx
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: my-nginx
spec:
  selector:
    matchLabels:
      run: my-nginx
  replicas: 1
  template:
    metadata:
      labels:
        run: my-nginx
    spec:
      volumes:
      - name: secret-volume
        secret:
          secretName: nginxsecret
      - name: configmap-volume
        configMap:
          name: nginxconfigmap
      containers:
      - name: nginxhttps
        image: bprashanth/nginxhttps:1.0
        ports:
        - containerPort: 443
        - containerPort: 80
        volumeMounts:
        - mountPath: /etc/nginx/ssl
          name: secret-volume
        - mountPath: /etc/nginx/conf.d
          name: configmap-volume

In [ ]:
kubectl delete deployments,svc my-nginx; kubectl create -f ./nginx-secure-app.yaml

In [ ]:
kubectl get pods -l run=my-nginx -o custom-columns=POD_IP:.status.podIPs
    POD_IP
    [map[ip:10.244.3.5]]

In [ ]:
node $ curl -k https://10.244.3.5
...
<h1>Welcome to nginx!</h1>

In [ ]:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: curl-deployment
spec:
  selector:
    matchLabels:
      app: curlpod
  replicas: 1
  template:
    metadata:
      labels:
        app: curlpod
    spec:
      volumes:
      - name: secret-volume
        secret:
          secretName: nginxsecret
      containers:
      - name: curlpod
        command:
        - sh
        - -c
        - while true; do sleep 1; done
        image: radial/busyboxplus:curl
        volumeMounts:
        - mountPath: /etc/nginx/ssl
          name: secret-volume

In [ ]:
kubectl apply -f ./curlpod.yaml
kubectl get pods -l app=curlpod

### Exposing the Service

In [ ]:
kubectl get svc my-nginx -o yaml | grep nodePort -C 5
  uid: 07191fb3-f61a-11e5-8ae5-42010af00002
spec:
  clusterIP: 10.0.162.149
  ports:
  - name: http
    nodePort: 31704
    port: 8080
    protocol: TCP
    targetPort: 80
  - name: https
    nodePort: 32453
    port: 443
    protocol: TCP
    targetPort: 443
  selector:
    run: my-nginx

In [ ]:
kubectl get nodes -o yaml | grep ExternalIP -C 1
    - address: 104.197.41.11
      type: ExternalIP
    allocatable:
--
    - address: 23.251.152.56
      type: ExternalIP
    allocatable:
...

$ curl https://<EXTERNAL-IP>:<NODE-PORT> -k
...
<h1>Welcome to nginx!</h1>

In [ ]:
kubectl edit svc my-nginx
kubectl get svc my-nginx

In [ ]:
curl https://<EXTERNAL-IP> -k
...
<title>Welcome to nginx!</title>

In [ ]:
kubectl describe service my-nginx
...
LoadBalancer Ingress:   a320587ffd19711e5a37606cf4a74574-1142138393.us-east-1.elb.amazonaws.com
...